In [119]:
# third party packages
import pandas as pd

In [146]:
# load the training data
data = pd.read_csv("data/train.csv")

In [121]:
def METADATA():
    return pd.DataFrame({
        'Type' : data.dtypes,
        'Null' : data.isnull().sum(),
        'Not Null' : data.notnull().sum(),
        'Unique' : data.nunique()
    }).sort_values(by=['Not Null'])

In [150]:
# initial metadata
display(METADATA())

,Type,Null,Not Null,Unique
NoOfExternalRef,float64,69377,71024,951
LineOfCode,float64,69152,71249,8311
HasSocialNet,float64,67997,72404,2
LargestLineLength,float64,67925,72476,17076
NoOfURLRedirect,float64,67383,73018,2
HasCopyrightInfo,float64,67343,73058,2
NoOfCSS,float64,67132,73269,178
NoOfObfuscatedChar,float64,66796,73605,8
NoOfSelfRedirect,float64,66715,73686,2
HasPasswordField,float64,66534,73867,2


In [148]:
# remove duplicate URL, while choosing not-null values for each feature
dupeURL = data['URL'].value_counts()
dupeURL = dupeURL[dupeURL > 1].index.to_list()

for URL in dupeURL:
    curData = data[data['URL'] == URL].copy() 
    for column in curData.columns:
        option = curData[column].dropna().unique()
        if len(option) > 0:
            data.loc[data['URL'] == URL, column] = option[0]
data.drop_duplicates(inplace=True)

In [149]:
# Fill URLLength
data.loc[data['URL'].notna() & data['URLLength'].isna(), 'URLLength'] = data['URL'].str.len()

# Fill Domain (improved extraction)
data.loc[data['URL'].notna() & data['Domain'].isna(), 'Domain'] = data['URL'].str.extract(r'https?://([^/]+)')[0]

# Fill DomainLength
data.loc[data['Domain'].notna() & data['DomainLength'].isna(), 'DomainLength'] = data['Domain'].str.len()

# Fill IsDomainIP
def IsIPCheck(obj):
    s = str(obj)
    ss = s.split('.')
    if len(ss) == 4 and all(part.isdigit() for part in ss):
        return 1
    return 0

data.loc[data['Domain'].notna() & data['IsDomainIP'].isna(), 'IsDomainIP'] = data['Domain'].apply(IsIPCheck)

# Fill NoOfSubDomain
data.loc[data['Domain'].notna() & data['NoOfSubDomain'].isna(), 'NoOfSubDomain'] = data['Domain'].str.split('.').str.len() - 2

# Fill NoOfQMarkInURL
def countQMark(obj):
    s = str(obj)
    count = 0
    for char in s:
        if char == '?': count += 1
    return count
data.loc[data['URL'].notna() & data['NoOfQMarkInURL'].isna(), 'NoOfQMarkInURL'] = data['URL'].apply(countQMark)